In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from scipy import stats
from numpy import inf
import networkx as nx
from dtaidistance import dtw
import random

#save and load dictionaries/lists
import pickle

#scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#PCA
from sklearn import decomposition

#clustering
from sklearn.cluster import KMeans #only numerical var
from sklearn.metrics import silhouette_samples, silhouette_score
import kmodes
from kmodes.kmodes import KModes #with categorical var as well

#plot
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import kmeans_clustering, ZoneVariable, time_series_henColumn_tsRow, FB_daily, corr_from_dep2feature,\
corr_from_feature2feature, correlationGraph, ZoneVariable, DataRepresentation1, sampen, chi2_distance, is_day, correct_key
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
id_run = config.id_run
dico_pen_tr = config.dico_pen_tr
li_binmn = config.li_binmn
penalty = config.penalty
dico_window = config.dico_window
birth_date = config.birth_date
dico_night_hour = config.dico_night_hour

change the configuration file if not done yet!


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

In [4]:
#download the cleaned-movement data
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', 
                 parse_dates=['Timestamp', 'date'], dayfirst=True) 
df['hour'] = df['Timestamp'].map(lambda x: x.hour)
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x-dt.timedelta(seconds=x.second)))
df.drop('duration', axis=1, inplace=True)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2059712, 17)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date,next_record_date,previous_record_date,previous_duration,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open,hour,time
0,hen_71,pen12,10 - 12,3_Zone,1,2020-09-29 09:07:00,2020-09-29,2020-09-29 09:08:26.000,NaN,NaN,3_Zone,NaN,NaN,False,False,9,09:07:00
1,hen_7,pen11,10 - 12,3_Zone,1,2020-09-29 09:08:12,2020-09-29,2020-09-29 09:12:16.000,NaN,NaN,3_Zone,NaN,NaN,False,False,9,09:08:00
2,hen_10,pen11,10 - 12,3_Zone,1,2020-09-29 09:19:19,2020-09-29,2020-09-29 09:20:27.000,NaN,NaN,3_Zone,NaN,NaN,False,False,9,09:19:00


# Compute MLPS vectors

### on the daily-hours - with list of zones per day per animal at second level

In [ ]:
#note that we will have more entries than needed, as the distrubances days are not removed in the raw-cleaned movements
#dataframe.
dico_pen_level_h = {}
#we have to do a loop per pen, due to memory issue
for p, df_pen in tqdm.tqdm(df.groupby('PenID')):
        
    #update results
    dico_pen_level_h[p] = {}
    
    #compute time series
    df_ts = time_series_henColumn_tsRow(df_pen, config, col_ts='Zone', ts_with_all_hen_value=False, save=False, 
                                        hen_time_series=False)
    
    #restrict to the daylight ONLY
    df_ts['is_day'] = df_ts['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
    df_ts = df_ts[df_ts['is_day']]

    #list of all hen present in this pen
    li_hen = [v for v in df_ts.columns if 'hen_' in v]

    ################# create one list per animal #################
    #groupby date to have a list of zones per day (rows) for the hens (columns)
    df_ts = df_ts.groupby('date')[li_hen].agg(lambda x: list(x)).reset_index()
    #melt to have one row per (day, hens) to avoid looping to create the dictionary
    df_ts_ = pd.melt(df_ts, id_vars=['date'], value_vars=li_hen)
    for d, df__ in df_ts_.groupby(['date']):
        #update results: value column is the list of zone
        dico_pen_level_h[p][d] = dict(zip(df__['HenID'].tolist(), df__['value'].tolist()))   
    
#save dictionaries
pickle.dump(dico_pen_level_h, open(os.path.join(path_extracted_data, 
                                                id_run+'dico_pen_level_h_allzoneidseclevel_DAILYLEVEL.pkl'), 'wb'), pickle.HIGHEST_PROTOCOL)

### on the daily-hours with bin(of duration in each bin)

In [5]:
#for efficiency purpose let's compute the bined time series first
#note that we will have more entries than needed, as the distrubances days are not removed in the raw-cleaned movements
#dataframe.
dico_pen_bin_zone_level_h = {}
dico_pen_bin_level_h = {}
def duration_normalized_perZone(x):
    c = Counter(x)
    t = len(x)
    return [c['1_Zone']/t, c['2_Zone']/t, c['3_Zone']/t, c['4_Zone']/t, c['5_Zone']/t]
#small example
#li = ['1_Zone','3_Zone','3_Zone','4_Zone','5_Zone','5_Zone','1_Zone']
#duration_normalized_perZone(li)

for p, df_pen in tqdm.tqdm(df.groupby('PenID')):
    
    #update results
    dico_pen_bin_zone_level_h[p] = {}
    dico_pen_bin_level_h[p] = {}
    
    #compute time series
    df_ts = time_series_henColumn_tsRow(df_pen, config, col_ts='Zone', ts_with_all_hen_value=False, save=False, 
                                        hen_time_series=False)
    
    for nbr_binmn in tqdm.tqdm(li_binmn):
        
        #update results
        dico_pen_bin_zone_level_h[p][nbr_binmn] = {}
        dico_pen_bin_level_h[p][nbr_binmn] = {}
                
        #reduce to the interval we want
        mi = min(df_ts['Timestamp'].tolist())
        ma = max(df_ts['Timestamp'].tolist())
        #extend the end to the end of the day in case it case the last day available fo the chicken
        Daterange = pd.date_range(start = mi, end = ma, freq = str(nbr_binmn)+'MIN')    
        df_date = pd.DataFrame({str(nbr_binmn)+'mn_timestamp':Daterange})
        new_timestamp = str(nbr_binmn)+'mn_timestamp'
        df_date[new_timestamp] = df_date[new_timestamp].map(lambda x: pd.to_datetime(x))
        df_ts_ = pd.merge_asof(df_ts, df_date, left_on=['Timestamp'], right_on=[new_timestamp], direction='forward')
        
        #restrict to the day ONLY
        df_ts_['is_day'] = df_ts_['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
        df_ts_ = df_ts_[df_ts_['is_day']]
        
        #groupby the interval that we want with the number of minutes in nestbox
        li_hen = [v for v in df_ts.columns if 'hen_' in v]
        
        ################# overall mlp #################
        df_sim = df_ts_.groupby(new_timestamp)[li_hen].agg(lambda x: duration_normalized_perZone(x)).reset_index()
        df_sim['date'] = df_sim[new_timestamp].map(lambda x: dt.datetime(x.year,x.month,x.day))
        #groupby date to have a list of zones per day (rows) for the hens (columns)
        df_sim = df_sim.groupby('date')[li_hen].agg(lambda x: list(x)).reset_index()
        #print(df_zone_sim.shape)
        #display(df_zone_sim.head(3))

        #melt to have one row per (day, hens) to avoid looping to create the dictionary
        df_sim_ = pd.melt(df_sim, id_vars=['date'], value_vars=li_hen)
        #variable column has the henIDs
        #print(df_zone_sim_.shape)
        #display(df_zone_sim_.head(3))
        for d, df__ in df_sim_.groupby(['date']):
            #update results
            dico_pen_bin_level_h[p][nbr_binmn][d] = dict(zip(df__['variable'].tolist(), df__['value'].tolist()))   
            
            
        ################# zone-ts over each zone #################                
        for ZONE in df['Zone'].unique():
            
            #update results
            dico_pen_bin_zone_level_h[p][nbr_binmn][ZONE] = {}
                
            df_zone_sim = df_ts_.groupby(new_timestamp)[li_hen].agg(lambda x: sum([i==ZONE for i in x])/60).reset_index()
            df_zone_sim['date'] = df_zone_sim[new_timestamp].map(lambda x: dt.datetime(x.year,x.month,x.day))

            #groupby date to have a list of zones per day (rows) for the hens (columns)
            df_zone_sim = df_zone_sim.groupby('date')[li_hen].agg(lambda x: list(x)).reset_index()
            #print(df_zone_sim.shape)
            #display(df_zone_sim.head(3))
            
            #melt to have one row per (day, hens) to avoid looping to create the dictionary
            df_zone_sim_ = pd.melt(df_zone_sim, id_vars=['date'], value_vars=li_hen)
            #variable column has the henIDs
            #print(df_zone_sim_.shape)
            #display(df_zone_sim_.head(3))
            for d, df__ in df_zone_sim_.groupby(['date']):
                #update results
                dico_pen_bin_zone_level_h[p][nbr_binmn][ZONE][d] = dict(zip(df__['variable'].tolist(), df__['value'].tolist()))
#save dictionaries
pickle.dump(dico_pen_bin_zone_level_h, open(os.path.join(path_extracted_data, 
                                                     id_run+'dico_pen_bin_zone_level_h_DAILYLEVEL.pkl'), 'wb'), pickle.HIGHEST_PROTOCOL)
pickle.dump(dico_pen_bin_level_h, open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_bin_level_h_DAILYLEVEL.pkl'), 'wb'), pickle.HIGHEST_PROTOCOL)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

in this time series there is 28 hens
The initial starting date in over all is: 2020-09-29 09:20:21, and the ending date will be: 2021-05-10 08:42:27
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:20:21, and the ending date will be: 2021-05-10 23:59:59



  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Total running time: 5.28 mn



 12%|█████████▊                                                                    | 1/8 [1:01:08<7:07:56, 3668.07s/it]

...


 75%|██████████████████████████████████████████████████████████▌                   | 6/8 [5:35:37<1:51:05, 3332.95s/it]

in this time series there is 29 hens
The initial starting date in over all is: 2020-09-29 09:52:12, and the ending date will be: 2021-05-10 08:40:33
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:52:12, and the ending date will be: 2021-05-10 23:59:59
Total running time: 5.12 mn



 88%|██████████████████████████████████████████████████████████████████████          | 7/8 [6:30:00<55:12, 3312.11s/it]

in this time series there is 28 hens
The initial starting date in over all is: 2020-09-29 09:36:51, and the ending date will be: 2021-05-10 08:39:39
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:36:51, and the ending date will be: 2021-05-10 23:59:59
Total running time: 3.96 mn



100%|████████████████████████████████████████████████████████████████████████████████| 8/8 [7:15:31<00:00, 3266.40s/it]


### on [2h, 17h59] only

In [6]:
#for efficiency purpose let's compute the bined time series first
#note that we will have more entries than needed, as the distrubances days are not removed in the raw-cleaned movements
#dataframe.
dico_pen_bin_zone_level_h = {}
dico_pen_bin_level_h = {}
def duration_normalized_perZone(x):
    c = Counter(x)
    t = len(x)
    return [c['1_Zone']/t, c['2_Zone']/t, c['3_Zone']/t, c['4_Zone']/t, c['5_Zone']/t]
#small example
#li = ['1_Zone','3_Zone','3_Zone','4_Zone','5_Zone','5_Zone','1_Zone']
#duration_normalized_perZone(li)

for p, df_pen in tqdm.tqdm(df.groupby('PenID')):
    
    #update results
    dico_pen_bin_zone_level_h[p] = {}
    dico_pen_bin_level_h[p] = {}
    
    #compute time series
    df_ts = time_series_henColumn_tsRow(df_pen, config, col_ts='Zone', ts_with_all_hen_value=False, save=False, 
                                        hen_time_series=False)
    
    for nbr_binmn in tqdm.tqdm(li_binmn):
        
        #update results
        dico_pen_bin_zone_level_h[p][nbr_binmn] = {}
        dico_pen_bin_level_h[p][nbr_binmn] = {}
                
        #reduce to the interval we want
        mi = min(df_ts['Timestamp'].tolist())
        ma = max(df_ts['Timestamp'].tolist())
        #extend the end to the end of the day in case it case the last day available fo the chicken
        Daterange = pd.date_range(start = mi, end = ma, freq = str(nbr_binmn)+'MIN')    
        df_date = pd.DataFrame({str(nbr_binmn)+'mn_timestamp':Daterange})
        new_timestamp = str(nbr_binmn)+'mn_timestamp'
        df_date[new_timestamp] = df_date[new_timestamp].map(lambda x: pd.to_datetime(x))
        df_ts_ = pd.merge_asof(df_ts, df_date, left_on=['Timestamp'], right_on=[new_timestamp], direction='forward')
        
        #restrict to 2h-17h59 ONLY
        df_ts_['is_2h17'] = df_ts_['Timestamp'].map(lambda x: (x.hour>=2)&(x.hour<18))
        df_ts_ = df_ts_[df_ts_['is_2h17']]
        
        #groupby the interval that we want with the number of minutes in nestbox
        li_hen = [v for v in df_ts.columns if 'hen_' in v]
        
        ################# overall mlp #################
        df_sim = df_ts_.groupby(new_timestamp)[li_hen].agg(lambda x: duration_normalized_perZone(x)).reset_index()
        df_sim['date'] = df_sim[new_timestamp].map(lambda x: dt.datetime(x.year,x.month,x.day))
        #groupby date to have a list of zones per day (rows) for the hens (columns)
        df_sim = df_sim.groupby('date')[li_hen].agg(lambda x: list(x)).reset_index()
        #print(df_zone_sim.shape)
        #display(df_zone_sim.head(3))

        #melt to have one row per (day, hens) to avoid looping to create the dictionary
        df_sim_ = pd.melt(df_sim, id_vars=['date'], value_vars=li_hen)
        #variable column has the henIDs
        #print(df_zone_sim_.shape)
        #display(df_zone_sim_.head(3))
        for d, df__ in df_sim_.groupby(['date']):
            #update results
            dico_pen_bin_level_h[p][nbr_binmn][d] = dict(zip(df__['variable'].tolist(), df__['value'].tolist()))   
            
            
        ################# zone-ts over each zone #################                
        for ZONE in df['Zone'].unique():
            
            #update results
            dico_pen_bin_zone_level_h[p][nbr_binmn][ZONE] = {}
                
            df_zone_sim = df_ts_.groupby(new_timestamp)[li_hen].agg(lambda x: sum([i==ZONE for i in x])/60).reset_index()
            df_zone_sim['date'] = df_zone_sim[new_timestamp].map(lambda x: dt.datetime(x.year,x.month,x.day))

            #groupby date to have a list of zones per day (rows) for the hens (columns)
            df_zone_sim = df_zone_sim.groupby('date')[li_hen].agg(lambda x: list(x)).reset_index()
            #print(df_zone_sim.shape)
            #display(df_zone_sim.head(3))
            
            #melt to have one row per (day, hens) to avoid looping to create the dictionary
            df_zone_sim_ = pd.melt(df_zone_sim, id_vars=['date'], value_vars=li_hen)
            #variable column has the henIDs
            #print(df_zone_sim_.shape)
            #display(df_zone_sim_.head(3))
            for d, df__ in df_zone_sim_.groupby(['date']):
                #update results
                dico_pen_bin_zone_level_h[p][nbr_binmn][ZONE][d] = dict(zip(df__['variable'].tolist(), df__['value'].tolist()))
#save dictionaries
pickle.dump(dico_pen_bin_zone_level_h, open(os.path.join(path_extracted_data, 
                                                     id_run+'dico_pen_bin_zone_level_h_2h-17h59LEVEL.pkl'), 'wb'), pickle.HIGHEST_PROTOCOL)
pickle.dump(dico_pen_bin_level_h, open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_bin_level_h_2h-17h59LEVEL.pkl'), 'wb'), pickle.HIGHEST_PROTOCOL)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

in this time series there is 28 hens
The initial starting date in over all is: 2020-09-29 09:20:21, and the ending date will be: 2021-05-10 08:42:27
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:20:21, and the ending date will be: 2021-05-10 23:59:59
Total running time: 3.61 mn



 12%|██████████                                                                      | 1/8 [38:53<4:32:11, 2333.13s/it]

in this time series there is 28 hens
The initial starting date in over all is: 2020-09-29 09:08:12, and the ending date will be: 2021-05-10 08:42:22
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:08:12, and the ending date will be: 2021-05-10 23:59:59
Total running time: 3.69 mn



 25%|███████████████████▌                                                          | 2/8 [1:17:54<3:53:33, 2335.50s/it]

...


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Total running time: 3.80 mn



 88%|██████████████████████████████████████████████████████████████████████          | 7/8 [4:42:58<40:55, 2455.80s/it]

in this time series there is 28 hens
The initial starting date in over all is: 2020-09-29 09:36:51, and the ending date will be: 2021-05-10 08:39:39
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:36:51, and the ending date will be: 2021-05-10 23:59:59
Total running time: 3.99 mn



100%|████████████████████████████████████████████████████████████████████████████████| 8/8 [5:26:47<00:00, 2450.97s/it]


### On the entire 24h period for generality

#for efficiency purpose let's compute the bined time series first
#note that we will have more entries than needed, as the distrubances days are not removed in the raw-cleaned movements
#dataframe.
dico_pen_bin_zone_level_h = {}
dico_pen_bin_level_h = {}
def duration_normalized_perZone(x):
    c = Counter(x)
    t = len(x)
    return [c['1_Zone']/t, c['2_Zone']/t, c['3_Zone']/t, c['4_Zone']/t, c['5_Zone']/t]
#small example
#li = ['1_Zone','3_Zone','3_Zone','4_Zone','5_Zone','5_Zone','1_Zone']
#duration_normalized_perZone(li)

for p, df_pen in tqdm.tqdm(df.groupby('PenID')):
    
    #update results
    dico_pen_bin_zone_level_h[p] = {}
    dico_pen_bin_level_h[p] = {}
    
    #compute time series
    df_ts = time_series_henColumn_tsRow(df_pen, config, col_ts='Zone', ts_with_all_hen_value=False, save=False, 
                                        hen_time_series=False)
    
    for nbr_binmn in tqdm.tqdm(li_binmn):
        
        #update results
        dico_pen_bin_zone_level_h[p][nbr_binmn] = {}
        dico_pen_bin_level_h[p][nbr_binmn] = {}
                
        #reduce to the interval we want
        mi = min(df_ts['Timestamp'].tolist())
        ma = max(df_ts['Timestamp'].tolist())
        #extend the end to the end of the day in case it case the last day available fo the chicken
        Daterange = pd.date_range(start = mi, end = ma, freq = str(nbr_binmn)+'MIN')    
        df_date = pd.DataFrame({str(nbr_binmn)+'mn_timestamp':Daterange})
        new_timestamp = str(nbr_binmn)+'mn_timestamp'
        df_date[new_timestamp] = df_date[new_timestamp].map(lambda x: pd.to_datetime(x))
        df_ts_ = pd.merge_asof(df_ts, df_date, left_on=['Timestamp'], right_on=[new_timestamp], direction='forward')
        #groupby the interval that we want with the number of minutes in nestbox
        li_hen = [v for v in df_ts.columns if 'hen_' in v]
        
        ################# overall mlp #################
        df_sim = df_ts_.groupby(new_timestamp)[li_hen].agg(lambda x: duration_normalized_perZone(x)).reset_index()
        df_sim['date'] = df_sim[new_timestamp].map(lambda x: dt.datetime(x.year,x.month,x.day))
        #groupby date to have a list of zones per day (rows) for the hens (columns)
        df_sim = df_sim.groupby('date')[li_hen].agg(lambda x: list(x)).reset_index()
        #print(df_zone_sim.shape)
        #display(df_zone_sim.head(3))

        #melt to have one row per (day, hens) to avoid looping to create the dictionary
        df_sim_ = pd.melt(df_sim, id_vars=['date'], value_vars=li_hen)
        #variable column has the henIDs
        #print(df_zone_sim_.shape)
        #display(df_zone_sim_.head(3))
        for d, df__ in df_sim_.groupby(['date']):
            #update results
            dico_pen_bin_level_h[p][nbr_binmn][d] = dict(zip(df__['variable'].tolist(), df__['value'].tolist()))   
            
            
        ################# zone-ts over each zone #################                
        for ZONE in df['Zone'].unique():
            
            #update results
            dico_pen_bin_zone_level_h[p][nbr_binmn][ZONE] = {}
                
            df_zone_sim = df_ts_.groupby(new_timestamp)[li_hen].agg(lambda x: sum([i==ZONE for i in x])/60).reset_index()
            df_zone_sim['date'] = df_zone_sim[new_timestamp].map(lambda x: dt.datetime(x.year,x.month,x.day))

            #groupby date to have a list of zones per day (rows) for the hens (columns)
            df_zone_sim = df_zone_sim.groupby('date')[li_hen].agg(lambda x: list(x)).reset_index()
            #print(df_zone_sim.shape)
            #display(df_zone_sim.head(3))
            
            #melt to have one row per (day, hens) to avoid looping to create the dictionary
            df_zone_sim_ = pd.melt(df_zone_sim, id_vars=['date'], value_vars=li_hen)
            #variable column has the henIDs
            #print(df_zone_sim_.shape)
            #display(df_zone_sim_.head(3))
            for d, df__ in df_zone_sim_.groupby(['date']):
                #update results
                dico_pen_bin_zone_level_h[p][nbr_binmn][ZONE][d] = dict(zip(df__['variable'].tolist(), df__['value'].tolist()))
#save dictionaries
pickle.dump(dico_pen_bin_zone_level_h, open(os.path.join(path_extracted_data, 
                                                     id_run+'dico_pen_bin_zone_level_h.pkl'), 'wb'), pickle.HIGHEST_PROTOCOL)
pickle.dump(dico_pen_bin_level_h, open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_bin_level_h.pkl'), 'wb'), pickle.HIGHEST_PROTOCOL)

#download the two dictionaries
dico_pen_bin_zone_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_bin_zone_level_h.pkl'), 'rb'))
dico_pen_bin_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                     id_run+'dico_pen_bin_level_h.pkl'), 'rb'))